Import the necessary libraries.

In [1]:
# !pip install pytextrank
# !pip install spacy
# !python -m spacy download en_core_web_sm

import pytextrank
import spacy
from icecream import ic
import re
import numpy as np

import nltk
from nltk import word_tokenize, pos_tag

from create_input_lists import lemmatize

2021-05-21 17:59:53.047297: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
     |████████████████████████████████| 13.7 MB 5.1 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
stop_words = 'all six eleven just less being indeed over both anyway detail four front already through yourselves\
 fifmill still its before move whose one system also somewhere herself thick show had enough should to onlyseeming\
 under herein ours two has might thereafter do them his around thereby get very de none cannotevery whether they not\
 during thus now him nor name regarding several hereafter did always cry whitherbeforehand this someone she each\
 further become thereupon where side towards few twelve because often tenanyhow doing km eg some back used go\
 namely besides yet are cant our beyond ourselves sincere out evenwhat throughout computer give for bottom mine\
 since please while per find everything behind does variousabove between kg neither seemed ever across t somehow\
 be we who were sixty however here otherwise whereupon nowhere although found hers re along quite fifteen by on\
 about didn last would anything via of could thence put against keep etc s became ltd hence therein onto or\
 whereafter con among own co afterwards formerly within seems into others whatever yourself down alone everyone done\
 least another whoever moreover couldnt must your three from her their together top there due been next anyone whom\
 much call too interest thru themselves hundred was until empty more himself elsewhere mostly that fire becomes\
 becoming hereby but else part everywhere former don with than those he me forty myself made full twenty these bill\
 using up us will nevertheless below anywhere nine can theirs toward my something and sometimes whenever sometime\
 then almost wherever is describe am it doesn an really as itself at have in seem whence ie any if again hasnt\
 inc un thin no perhaps latter meanwhile when amount same wherein beside how other take which latterly you\
 fill either nobody unless whereas see though may after upon therefore most hereupon eight amongst never\
 serious nothing such why a off whereby third i whole noone many well except amoungst yours rather without\
 so five the first having once'

custom_stopwords = ['correct', 'true', 'false', 'yes', 'following', 'yellow']
stop_words = word_tokenize(stop_words)
stop_words = stop_words + custom_stopwords
print(stop_words)

['all', 'six', 'eleven', 'just', 'less', 'being', 'indeed', 'over', 'both', 'anyway', 'detail', 'four', 'front', 'already', 'through', 'yourselves', 'fifmill', 'still', 'its', 'before', 'move', 'whose', 'one', 'system', 'also', 'somewhere', 'herself', 'thick', 'show', 'had', 'enough', 'should', 'to', 'onlyseeming', 'under', 'herein', 'ours', 'two', 'has', 'might', 'thereafter', 'do', 'them', 'his', 'around', 'thereby', 'get', 'very', 'de', 'none', 'cannotevery', 'whether', 'they', 'not', 'during', 'thus', 'now', 'him', 'nor', 'name', 'regarding', 'several', 'hereafter', 'did', 'always', 'cry', 'whitherbeforehand', 'this', 'someone', 'she', 'each', 'further', 'become', 'thereupon', 'where', 'side', 'towards', 'few', 'twelve', 'because', 'often', 'tenanyhow', 'doing', 'km', 'eg', 'some', 'back', 'used', 'go', 'namely', 'besides', 'yet', 'are', 'cant', 'our', 'beyond', 'ourselves', 'sincere', 'out', 'evenwhat', 'throughout', 'computer', 'give', 'for', 'bottom', 'mine', 'since', 'please', 

In [16]:
def simple_preprocessText(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z-\s]','', text)
    return text

In [17]:
# clusteredText = 'Cytosine: pyrimidine Correct _ This is a pyrimidine.. Thymine: pyrimidine Correct _ Thymine is a pyrimidine.. Uracil: pyrimidine Correct _ Uracil is a pyrimidine..'
clusteredText = 'Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page.. Return to the top of the page..'
clusteredText = 'Upon completing this module, students will be able to: . After examing atoms and their properties, students will be able to:. By the end of this exploration of water, students will be able to:. By the end of this module, students should be able to:. By the end of this module, students should be able to:. By the end of this module, students should be able to:. By the end of this module, students should be able to:. By the end of this module, students should be able to:. By the end of this module, students should be able to:. Upon completion of this module, students will be able to:. Upon completion of this module, students will be able to:. Upon investigation of transport proteins, students will be able to:. After studying this introduction to endocytosis, students will be able to:. Upon further exploratiion of endocytosis, students will be able to: .'
clusteredText = 'You can take this quiz only once _ .  The following quiz will be graded and recorded in the grade book _ You may take this quiz only once..  The following quiz will be graded and recorded in the grade book _ You may take this quiz only once..  The following quiz will be graded and recorded in the grade book _ You may take this quiz only once..  The following quiz will be graded and recorded in the grade book _ You may take this quiz only once..  The following quiz will be graded and recorded in the grade book _ You may take this quiz only once.. '
clusteredText = 'A structured and active process used to test a hypothesis _ In an experiment, the scientist has some form of control over the variables at work in the system under study _ Typically some variable is deliberately changed by the experimenter and other variables are monitored to see if they change as a result.. A structured and active process used to test a hypothesis _ In an experiment, the scientist has some form of control over the variables at work in the system under study _ Typically some variable is deliberately changed by the experimenter and other variables are monitored to see if they change as a result..  '
clusteredText = 'A group of similar cells carrying out the same detailed function within an organism _ In the lung, for instance, the lining of airways comprises one type of tissue: a layer of similar cells that forms a sealed surface _ . A groups of similar cells carrying out the same detailed function within an organism _ In the lung, for instance, the lining of airways comprises one type of tissue: a layer of similar cells that forms a sealed surface _ . '
clusteredText = 'Given that Mendel’s cross between these two true-breeding plants resulted in an F1 generation that all produced yellow round seeds, which of the following statements is true? Yellow and round are dominant traits; green and wrinkled are recessive traits _ Correct _ The offspring all have round yellow seeds, which means both these traits must be coded for by dominant alleles _ The parent phenotypes are “yellow round” and “green wrinkled.” All the offspring from these parents are “yellow round.”.  yellow,round 9 Correct _ Y (yellow) and R (round) are the dominant alleles _ Therefore any individual with at least one Y will be yellow and any individual with at least one R will be round _ Y (yellow) and R (round) are both dominant alleles _ Any individual with at least one Y will be yellow in color and any individual with at least one R will be round..  yellow, wrinkled 3 Correct _ Y (yellow) is the dominant allele and r (wrinkled) is the recessive allele _ Therefore any individual with at least one Y will be yellow, but an individual must have two r\'s (rr) to be wrinkled _ Y (yellow) is a dominant allele and r (wrinkled) is a recessive allele _ If an individual has at least one Y it will be yellow in color _ An individual has to have two r\'s (rr) to be wrinkled..  green, round 3 Correct _ y (green) is the recessive allele and R (round) is the dominant allele _ Therefore an individual must have two y\'s (yy) to be green and it will be round with at least one R allele _ y (green) is a recessive allele and R (round) is a dominant allele _ An individual must have two y\'s (yy) to be green in color _ An individual having at least on R will be round..  green, wrinkled 1 Correct _ Both y (green) and r (wrinkled) are recessive alleles _ Therefore an individual must have two y\'s (yy) to be green and two r\'s (rr) to be wrinkled _ Both y (green) and r (wrinkled) are recessive alleles _ An individual must have two y\'s (yy) to be green _ An individual must have two r\'s (rr) to be wrinkled..  '

lemmatized_clusteredText = lemmatize(clusteredText)
print(lemmatized_clusteredText, '\n')

clusteredText = simple_preprocessText(clusteredText)

original_words = word_tokenize(clusteredText)



lemmas = word_tokenize(lemmatized_clusteredText)

more_custom_stopwords = []

for word in np.unique(original_words):
    if word not in np.unique(lemmas):
        more_custom_stopwords.append(word)
print(more_custom_stopwords)

stop_words = stop_words + more_custom_stopwords
        
print('Original Text:', clusteredText, '\n')

parts_of_speech = ['ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PROPN', 'PRON', 'PART', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']

stopword_dict = {}
for word in stop_words:
    stopword_dict[word] = parts_of_speech
    


give mendel cross breeding plant result f generation produce yellow round seed statement yellow round dominant trait green wrinkle recessive trait offspring round yellow seed mean trait cod dominant allele parent phenotype yellow round green wrinkle offspring parent yellow round yellow round y yellow r round dominant allele individual y yellow individual r round y yellow r round dominant allele individual y yellow color individual r round yellow wrinkle y yellow dominant allele r wrinkle recessive allele individual y yellow individual r rr wrinkle y yellow dominant allele r wrinkle recessive allele individual y yellow color individual r rr wrinkle green round y green recessive allele r round dominant allele individual y yy green round r allele y green recessive allele r round dominant allele individual y yy green color individual r round green wrinkle y green r wrinkle recessive allele individual y yy green r rr wrinkle y green r wrinkle recessive allele individual y yy green individua

In [20]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank", config={'pos_kept': ['ADJ', 'NOUN'], 'stopwords': stopword_dict, 'token_lookback': 2})
nlp.analyze_pipes(pretty=True)
doc = nlp(clusteredText)


============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   tok2vec           doc.tensor                                          False      
                                                                                     
1   tagger            token.tag                        tag_acc            False      
                                                                                     
2   parser            token.dep                        dep_uas            False      
                      token.head                       dep_las                       
                      token.is_sent_start              dep_las_per_type              
                      doc.sents                        sents_p                       
                                                       sents

Add the PyTextRank component into the spac

Examine the top-ranked phrases in the document.

In [21]:
for phrase in doc._.phrases:
    ic(phrase.rank, phrase.count, phrase.text)
    ic(phrase.chunks)

ic| phrase.rank: 0.19083984615349237
    phrase.count: 1
    phrase.text: 'round yellow seeds'
ic| phrase.chunks: [round yellow seeds]
ic| phrase.rank: 0.19083984615349237
    phrase.count: 1
    phrase.text: 'yellow round seeds'
ic| phrase.chunks: [yellow round seeds]
ic| phrase.rank: 0.1813537953416753
    phrase.count: 2
    phrase.text: 'dominant alleles'
ic| phrase.chunks: [dominant alleles, dominant alleles]
ic| phrase.rank: 0.1759994281645476
    phrase.count: 2
    phrase.text: 'recessive alleles'
ic| phrase.chunks: [recessive alleles, recessive alleles]
ic| phrase.rank: 0.14642353430816057
    phrase.count: 1
    phrase.text: 'r round'
ic| phrase.chunks: [r round]
ic| phrase.rank: 0.12788369487307963
    phrase.count: 1
    phrase.text: 'recessive traits'
ic| phrase.chunks: [recessive traits]
ic| phrase.rank: 0.12178634785380807
    phrase.count: 5
    phrase.text: 'r'
ic| phrase.chunks: [r, r, r, r, r]
ic| phrase.rank: 0.09617737386001195
    phrase.count: 2
    phrase.text: 

In [13]:
print(doc.text)

give mendel cross breeding plant result f generation produce yellow round seed statement yellow round dominant trait green wrinkle recessive trait offspring round yellow seed mean trait cod dominant allele parent phenotype yellow round green wrinkle offspring parent yellow round yellow round y yellow r round dominant allele individual y yellow individual r round y yellow r round dominant allele individual y yellow color individual r round yellow wrinkle y yellow dominant allele r wrinkle recessive allele individual y yellow individual r rr wrinkle y yellow dominant allele r wrinkle recessive allele individual y yellow color individual r rr wrinkle green round y green recessive allele r round dominant allele individual y yy green round r allele y green recessive allele r round dominant allele individual y yy green color individual r round green wrinkle y green r wrinkle recessive allele individual y yy green r rr wrinkle y green r wrinkle recessive allele individual y yy green individua